<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220330_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 3.8 MB 81.4 MB/s 
     |████████████████████████████████| 1.1 MB 71.3 MB/s 
     |████████████████████████████████| 134 kB 75.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 92.8 MB/s 
     |████████████████████████████████| 127 kB 80.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 83.7 MB/s 
     |████████████████████████████████| 144 kB 77.3 MB/s 
     |████████████████████████████████| 596 kB 77.6 MB/s 
     |████████████████████████████████| 895 kB 79.3 MB/s 
     |████████████████████████████████| 6.5 MB 56.4 MB/s 
     |████████████████████████████████| 1.2 MB 59.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [9]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [6]:
df = df.fillna(0)

In [10]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱,C 13 134
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스,P 85 856


In [11]:
train = df[['text', 'target']]
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...
999995,제품입고 워싱 청바지워싱,C 13 134
999996,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,사업장에서 일반고객을대상으로 필라테스,P 85 856


# 토크나이저, 함수

In [12]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [13]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [14]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,0
1,상점내에서 일반인을 대상으로 채소.과일판매,1
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2
3,영업점에서 일반소비자에게 열쇠잠금장치,3
4,어린이집 보호자의 위탁을 받아 취학전아동보육,4
...,...,...
999995,제품입고 워싱 청바지워싱,103
999996,현장에서 고객의요청에의해 실내인테리어,68
999997,영업점에서 일반소비자에게 여성의류 판매,11
999998,사업장에서 일반고객을대상으로 필라테스,12


In [40]:
my_dict['0.0 0.0 0.0'] = len(my_dict)

In [15]:
train[train['text'].isnull()]

,text,target


In [16]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [17]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 5
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/SANUP/model' + str(fold), save_function=accelerator.save)

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/15625 [00:00<?, ?it/s]

  0%|          | 0/15625 [00:00<?, ?it/s]

  0%|          | 0/15625 [00:00<?, ?it/s]

...

  0%|          | 0/15625 [00:00<?, ?it/s]

  0%|          | 0/15625 [00:00<?, ?it/s]

  0%|          | 0/15625 [00:00<?, ?it/s]

  0%|          | 0/15625 [00:00<?, ?it/s]

epoch 1 train acc 0.7084
epoch 1 train acc 0.7075
epoch 1 train acc 0.70643
epoch 1 train acc 0.70663
epoch 1 train acc 0.70721
epoch 1 train acc 0.70527
epoch 1 train acc 0.70756
epoch 1 train acc 0.70797
epoch 1 test acc 0.865888746803069
epoch 1 test acc 0.8731218030690537
epoch 1 test acc 0.8686460997442456
epoch 1 test acc 0.8664881713554987
epoch 1 test acc 0.8686460997442456
epoch 1 test acc 0.8668478260869565
epoch 1 test acc 0.8652093989769821
epoch 1 test acc 0.86880594629156
epoch 2 train acc 0.87826
epoch 2 train acc 0.87706
epoch 2 train acc 0.87987
epoch 2 train acc 0.8773
epoch 2 train acc 0.87783
epoch 2 train acc 0.8785
epoch 2 train acc 0.87761
epoch 2 train acc 0.87798
epoch 2 test acc 0.8957800511508951
epoch 2 test acc 0.8955402813299232
epoch 2 test acc 0.8978980179028133
epoch 2 test acc 0.894781010230179
epoch 2 test acc 0.8956202046035806
epoch 2 test acc 0.8949808184143222
epoch 2 test acc 0.8940217391304348
epoch 2 test acc 0.8930226982097187
epoch 3 train ac

In [17]:
print(my_variable)

NameError: ignored

In [26]:
test = submission.fillna(0)

In [27]:
test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,0.0,0.0,0.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,0.0,0.0,0.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,0.0,0.0,0.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,0.0,0.0,0.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,0.0,0.0,0.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,0.0,0.0,0.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,0.0,0.0,0.0,한의원에서,외래환자위주고,치료
99997,id_099998,0.0,0.0,0.0,일반점포에서,소비자에게,그림판매
99998,id_099999,0.0,0.0,0.0,사업장에서,일반인.학생대상으로,학습공간제공


In [28]:
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_000001,0.0,0.0,0.0,치킨전문점에서,고객의주문에의해,치킨판매,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0
1,id_000002,0.0,0.0,0.0,산업공구,다른 소매업자에게,철물 수공구,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0
2,id_000003,0.0,0.0,0.0,절에서,신도을 대상으로,불교단체운영,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0
3,id_000004,0.0,0.0,0.0,영업장에서,고객요구로,자동차튜닝,영업장에서 고객요구로 자동차튜닝,0.0 0.0 0.0
4,id_000005,0.0,0.0,0.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공","실내포장마차에서 접객시설을 갖추고 소주,맥주제공",0.0 0.0 0.0
...,...,...,...,...,...,...,...,...,...
99995,id_099996,0.0,0.0,0.0,사업장에서,일반인대상으로,버섯농장,사업장에서 일반인대상으로 버섯농장,0.0 0.0 0.0
99996,id_099997,0.0,0.0,0.0,한의원에서,외래환자위주고,치료,한의원에서 외래환자위주고 치료,0.0 0.0 0.0
99997,id_099998,0.0,0.0,0.0,일반점포에서,소비자에게,그림판매,일반점포에서 소비자에게 그림판매,0.0 0.0 0.0
99998,id_099999,0.0,0.0,0.0,사업장에서,일반인.학생대상으로,학습공간제공,사업장에서 일반인.학생대상으로 학습공간제공,0.0 0.0 0.0


In [29]:
test = test[['text', 'target']]
test

,text,target
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0
3,영업장에서 고객요구로 자동차튜닝,0.0 0.0 0.0
4,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공",0.0 0.0 0.0
...,...,...
99995,사업장에서 일반인대상으로 버섯농장,0.0 0.0 0.0
99996,한의원에서 외래환자위주고 치료,0.0 0.0 0.0
99997,일반점포에서 소비자에게 그림판매,0.0 0.0 0.0
99998,사업장에서 일반인.학생대상으로 학습공간제공,0.0 0.0 0.0


In [50]:
test

,text,target
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0
3,영업장에서 고객요구로 자동차튜닝,0.0 0.0 0.0
4,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공",0.0 0.0 0.0
...,...,...
99995,사업장에서 일반인대상으로 버섯농장,0.0 0.0 0.0
99996,한의원에서 외래환자위주고 치료,0.0 0.0 0.0
99997,일반점포에서 소비자에게 그림판매,0.0 0.0 0.0
99998,사업장에서 일반인.학생대상으로 학습공간제공,0.0 0.0 0.0


# 추론

In [56]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/SANUP/model' + str(fold), num_labels=225)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/SANUP/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/SANUP/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

In [57]:
pred = pd.read_csv('/content/drive/MyDrive/SANUP/pred_label4.csv')
pred

,index,label
0,0,I 56 561
1,1,G 46 466
2,2,S 94 949
3,3,S 95 952
4,4,I 56 562
...,...,...
99995,99995,G 47 472
99996,99996,Q 86 862
99997,99997,G 47 478
99998,99998,R 90 902


In [81]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [82]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = pred['label'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = pred['label'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = pred['label'].map(split_label)

In [83]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,47,472,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공


In [84]:
submission.to_csv('/content/drive/MyDrive/SANUP/submission.csv', index=False)

In [ ]:
print(my_variable)

## hardvoting

### voting algorithm 
  + <strong> 5개의 모델이 판단한 5개의 label 중 동일한 label이 3개 이상이면 해당 label로 분류, 그 외의 경우는 neutral로 분류
  
  + 앞선 과정을 거쳐 neutral로 분류된 데이터 중 5개의 label이 neutral과 entailment, <br> neutral과 contradiction이 경합하면 entailment와 contradiction으로 재분류 </strong>

In [85]:
pred0 = pd.read_csv('/content/drive/MyDrive/SANUP/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/SANUP/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/SANUP/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/SANUP/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/SANUP/pred_label4.csv')

In [86]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [87]:
df.head(3)

,text,target,label0,label1,label2,label3,label4
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949


In [111]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

0.91963

In [108]:
df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]

,text,target,label0,label1,label2,label3,label4
9,"약품(화공), 미싱 완성품입고, 수선 0",0.0 0.0 0.0,C 25 259,C 25 259,C 25 259,C 34 340,C 34 340
21,매니큐어용기 0 0,0.0 0.0 0.0,G 46 464,G 46 464,C 13 132,C 13 132,G 47 475
38,"PP.PET 원료투입, 압출 원사",0.0 0.0 0.0,C 22 222,C 22 222,C 22 222,C 22 222,C 13 139
44,"제조업 업자를 대상으로 칫솔,부러쉬",0.0 0.0 0.0,C 27 271,C 27 271,C 27 271,C 20 204,C 32 320
49,매장에서 일반소비자에게 식료품소매1,0.0 0.0 0.0,G 47 471,G 47 471,G 47 471,G 47 471,G 47 472
...,...,...,...,...,...,...,...
99953,사업장에서 철제로 부직포기계,0.0 0.0 0.0,C 29 292,C 29 292,C 29 292,C 25 259,C 25 259
99956,사무실에서 교량 택지관련사업장의의뢰를 받아 계측자동화시스템개발,0.0 0.0 0.0,J 62 620,J 62 620,J 62 620,J 62 620,M 70 701
99957,음식점에서 접객시설을 갖추고 곱창밑 주류판매,0.0 0.0 0.0,I 56 562,I 56 561,I 56 562,I 56 562,I 56 562
99979,업주들을대상으로 사무실에서 검사및회계사무지원,0.0 0.0 0.0,M 71 715,M 71 715,M 71 715,M 71 711,M 71 715


* 모델의 예측 결과가 neutral에 치중되는 경향을 보임
  - label 별로 threshold를 다르게 해야 할 필요가 있음

In [ ]:
num_contradiction = len(df[df['label0'] == 'contradiction']) + len(df[df['label1'] == 'contradiction']) + len(df[df['label2'] == 'contradiction']) + len(df[df['label3'] == 'contradiction']) + len(df[df['label4'] == 'contradiction'])
num_entailment = len(df[df['label0'] == 'entailment']) + len(df[df['label1'] == 'entailment']) + len(df[df['label2'] == 'entailment']) + len(df[df['label3'] == 'entailment']) + len(df[df['label4'] == 'entailment'])
num_neutral = len(df[df['label0'] == 'neutral']) + len(df[df['label1'] == 'neutral']) + len(df[df['label2'] == 'neutral']) + len(df[df['label3'] == 'neutral']) + len(df[df['label4'] == 'neutral'])

labels = ['contradiction', 'entailment', 'neutral']
counts = [num_contradiction, num_entailment, num_neutral]

plt.bar(labels, counts)
plt.show()

In [ ]:
def label_count(df):
  num_neutral = 0
  num_contradiction = 0
  num_entailment = 0
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    if df[col] == 'neutral':
      num_neutral = num_neutral + 1
    if df[col] == 'contradiction':
      num_contradiction = num_contradiction + 1
    if df[col] == 'entailment':
      num_entailment = num_entailment + 1

  return [num_neutral, num_contradiction, num_entailment]

temp = df.apply(label_count, axis=1)

df['temp'] = temp

df['neutral'] = 0
df['contradiction'] = 0
df['entailment'] = 0

def list_to_num(list:list):
  return list[0]
df['neutral'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[1]
df['contradiction'] = df['temp'].map(list_to_num)

def list_to_num(list:list):
  return list[2]
df['entailment'] = df['temp'].map(list_to_num)

def voting(df):
  cols = ['neutral', 'contradiction', 'entailment']
  for col in cols:
    if df[col] > 2:
      return col
  return 'neutral'

df['label'] = df.apply(voting, axis=1)

- 동일한 label이 3개 이상이면 해당 label로 분류
- 그 외 neutral

In [ ]:
df.head(3)

- label이 neutral에 지나치게 치중되는 경향을 보임

In [ ]:
num_contradiction = df['label'].value_counts()['contradiction']
num_entailment = df['label'].value_counts()['entailment']
num_neutral = df['label'].value_counts()['neutral']

counts = [num_contradiction, num_entailment, num_neutral]

plt.bar(labels, counts)
plt.show()

- label이 neutral인 데이터 중 entailment와 contradiction로 분류될 가능성이 있는 데이터의 label 조정
  + neutral: 3 / contradiction: 2 -> **contradiction**
  + neutral: 3 / entailment: 2 -> **entailment**
  + neutral: 4 /  contradiction: 1 -> **contradiction**
  + neutral: 4 / entailment: 1 -> **entailment**

In [ ]:
def make_label(list:list):
  if list == [3, 2, 0] :
    return 'contradiction'
  if list == [3, 0, 2]:
    return 'entailment'
  if list == [4, 1, 0]:
    return 'contradiction'
  if list == [4, 0, 1]:
    return 'entailment'
  else :
    return 'answer'

my_label = df['temp'].map(make_label)
df['my_label'] = my_label
my_answer = df[df['my_label'] != 'answer']
df.loc[my_answer.index, 'label'] = my_answer['my_label']

In [ ]:
num_contradiction = df['label'].value_counts()['contradiction']
num_entailment = df['label'].value_counts()['entailment']
num_neutral = df['label'].value_counts()['neutral']

counts = [num_contradiction, num_entailment, num_neutral]

plt.bar(labels, counts)
plt.show()

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [ ]:
submission['label'] = df['label']

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/sm_220228/submission_hard.csv', index=False)

## softvoting

###  voting algorithm
  + <strong> 동일한 premise에 세 가지의 hypothesis가 매칭되는 경우, 가장 높은 확률을 갖는 hypothesis부터 label 부여  
  + 동일한 premise에 세 가지의 hypothesis가 매칭되지 않는 경우, 가장 큰 확률을 갖는 label 부여 </strong>

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/sm_220228/pred_prob4.csv')

In [ ]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

* 모델의 예측 결과가 neutral에 치중되는 경향을 보임
  - label 별로 threshold를 다르게 해야 할 필요가 있음

In [ ]:
prob_entailment = test[0].mean()
prob_contradiction = test[1].mean()
prob_neutral = test[2].mean()

probs = [prob_contradiction, prob_entailment, prob_neutral]

plt.bar(labels, probs)
plt.show()

In [ ]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['label'] = pd.DataFrame(answer)[1]
test.head(3)

- 가장 큰 확률을 갖는 label을 부여하면 hardvoting과 같이 neutral에 치중되는 경향을 보임

- 분류된 데이터 중 5개의 label이 neutral과 entailment, neutral과 contradiction이 경합하면 entailment와 contradiction으로 재분류

In [ ]:
num_contradiction = test['label'].value_counts()['contradiction']
num_entailment = test['label'].value_counts()['entailment']
num_neutral = test['label'].value_counts()['neutral']

counts = [num_contradiction, num_entailment, num_neutral]

plt.bar(labels, counts)
plt.show()

- 보편적으로 하나의 premise에 세 가지의 hypothesis가 매칭

In [ ]:
test.sort_values(by='premise')[:15]

In [ ]:
for i, sent in enumerate(test['premise'].unique()):
  if len(test[test['premise'] == sent]) == 3:
    continue
  else :
    print(sent)

In [ ]:
test[test['premise'] == '한겨례평화통일포럼은 18일 안산시 드림스타트 아동들에게 전달해 달라며 쌀 48포를 기탁했다.']

- 동일한 premise에 세 가지의 hypothesis가 매칭되는 경우, 가장 높은 확률을 갖는 hypothesis부터 label 부여
- 남은 hypothesis는 앞서 선정된 label을 제외한 두가지의 label 중 높은 확률을 갖는 label 부여
- 동일한 premise에 세 가지의 hypothesis가 매칭되지 않는 경우, 가장 큰 확률을 갖는 label 부여

In [ ]:
def find_max(test):
  return max(test[0], test[1], test[2])

test['max'] = test.apply(find_max, axis=1)
test.sort_values(by='premise').head(3)

In [ ]:
for i, sent in enumerate(test['premise'].unique()):
  labels = [0,1,2]
  if len(test[test['premise'] == sent]) == 3:
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0]['index']), 'label'] = test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels])]
    labels.remove(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels])])
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1]['index']), 'label'] = test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels])]
    labels.remove(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels].index[np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[1][labels])])
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[2]['index']), 'label'] = labels[0]
  else :
    test.loc[(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0]['index']), 'label'] = np.argmax(test[test['premise'] == sent].sort_values(by='max', ascending=False).iloc[0][labels])

In [ ]:
test['label'] = pd.DataFrame(num_to_label(test['label']))[1]
test.head(3)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/KLUE/sample_submission.csv')

In [ ]:
submission['label'] = test['label']

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/sm_220228/submission_soft.csv', index=False)